In [2]:
import sys
sys.path.append('..')

from src.tacogfn.utils.molecules import evaluate_properties2

import torch
import numpy as np
from tqdm import tqdm

import rdkit
rdkit.RDLogger.DisableLog('rdApp.*')

/home/tsa87/anaconda3/envs/tacogfn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
generated_store = torch.load('../logs/finetune/generated_molecules.pt')

In [4]:
generated_ds_exp_store = torch.load('../logs/finetune-1.5docking_score2/generated_molecules_139.pt')

In [5]:
generated_replay_online_store = torch.load('../logs/finetune-R-PRS-WithOnline/generated_molecules_98.pt')

In [6]:
generated_bs32_store = torch.load('../logs/finetune-bs-32/generated_molecules_687.pt')

In [7]:
generated_replay_store = torch.load('../logs/finetune-R-PRS/generated_molecules_160.pt')

In [8]:
no_finetune_ds = torch.load('../misc/ranking_analysis/14gs_A_docking_scores_no_finetune.pt')

In [9]:
generated_high_beta = torch.load('../logs/finetune-1.5docking_score2-high-beta/generated_molecules_206.pt')

In [10]:
generated_high_qed = torch.load('../logs/finetune-1.5docking_score2-qed/2/generated_molecules_50.pt')

In [11]:
generated_high_qed_beta = torch.load('../logs/finetune-1.5docking_score2-qed-high-beta/2/generated_molecules_46.pt')

In [12]:
top_count = 100

In [13]:
def get_top_molecules_across_count(generated_store, top_count=100, limit=25000):
    top_100 = []
    
    docking_scores = np.array([ds for ds in generated_store['docking_scores']])[:limit]
    smiles = np.array(generated_store['generated_molecules'])[:limit]

    for i in range(top_count, len(docking_scores), 100):
        subset = docking_scores[:i]
        sorted_idx = np.argsort(subset)
        top_100.append(np.mean(subset[sorted_idx][:top_count]))

    top_100_smiles = smiles[sorted_idx[:top_count]]
    metrics = evaluate_properties2(top_100_smiles)
    metrics['docking_scores'] = np.mean(top_100[-1])
    
    return top_100, metrics

In [14]:
top_100, top_100_metrics = get_top_molecules_across_count(generated_store)
top_100_metrics

{'qeds': 0.4094917202036973,
 'sas': 0.7060566686792782,
 'diversity': 0.6237563057574979,
 'docking_scores': -9.87606}

In [15]:
import random
top_100_no_finetune = []

random.shuffle(no_finetune_ds)
for i in range(top_count, len(no_finetune_ds), 100):
    subset = no_finetune_ds[:i]
    subset.sort()
    top_100_no_finetune.append(np.mean(subset[:top_count]))

In [16]:
top_100_bs32, top_100_bs32_metrics = get_top_molecules_across_count(generated_bs32_store)
top_100_bs32_metrics

{'qeds': 0.4187845731866451,
 'sas': 0.7196864381962894,
 'diversity': 0.6187608272990914,
 'docking_scores': -9.73468}

In [17]:
top_100_replay, top_100_replay_metrics = get_top_molecules_across_count(generated_replay_store)
top_100_replay_metrics

{'qeds': 0.3323523074071874,
 'sas': 0.6787609322391941,
 'diversity': 0.5942186505029039,
 'docking_scores': -9.73337999343872}

In [18]:
top_100_replay_online, top_100_replay_online_metrics = get_top_molecules_across_count(generated_replay_online_store)
top_100_replay_online_metrics

{'qeds': 0.3466846113300797,
 'sas': 0.6599608937589677,
 'diversity': 0.5991789877018514,
 'docking_scores': -9.765779933929444}

In [19]:
top_100_ds_exp, top_100_ds_exp_metrics = get_top_molecules_across_count(generated_ds_exp_store)
top_100_ds_exp_metrics

{'qeds': 0.40687138194471584,
 'sas': 0.7494287605353729,
 'diversity': 0.6134469118390927,
 'docking_scores': -9.927690010070801}

In [20]:
top_100_ds_exp_beta, top_100_ds_exp_beta_metrics = get_top_molecules_across_count(generated_high_beta)
top_100_ds_exp_beta_metrics 

{'qeds': 0.5105085241599544,
 'sas': 0.79713028735483,
 'diversity': 0.6053654799735978,
 'docking_scores': -9.909999952316284}

In [21]:
top_100_high_qed, top_100_high_qed_metrics = get_top_molecules_across_count(generated_high_qed)
top_100_high_qed_metrics

{'qeds': 0.4797512175747641,
 'sas': 0.6976768597921683,
 'diversity': 0.6178823794956047,
 'docking_scores': -9.62077998161316}

In [22]:
top_100_high_qed_beta, top_100_high_qed_beta_metrics = get_top_molecules_across_count(generated_high_qed_beta)
top_100_high_qed_beta_metrics

{'qeds': 0.612554113488296,
 'sas': 0.7177881963710466,
 'diversity': 0.6277631274207905,
 'docking_scores': -9.710739974975587}

: 

In [23]:
import matplotlib.pyplot as plt

plt.plot(top_100, label='With Fine-tuning (Batch Size=64)')
plt.plot(top_100_no_finetune, label='Without Fine-tuning')
plt.plot(top_100_bs32, label='With Fine-tuning (Batch Size=32)')
plt.plot(top_100_replay, label='With Fine-tuning (Priority Replay Buffer)')
plt.plot(top_100_replay_online, label='With Fine-tuning (Online and Priority Replay Buffer)')
plt.plot(top_100_ds_exp, label='With Fine-tuning (Docking score exp)')
plt.plot(top_100_ds_exp_beta, label='With Fine-tuning (Docking score exp beta)')
plt.plot(top_100_high_qed, label='With Fine-tuning (QED)')
plt.plot(top_100_high_qed_beta, label='With Fine-tuning (QED beta)')
plt.ylabel('Top 100 Docking Score')
plt.xlabel('Number of Molecules fine-tuned (*100)')
plt.legend()
plt.show()


In [37]:
import json

In [40]:
file_path = '../misc/generated_molecules/20240512_1.0_1.0_1000_finetune-1.5docking_score2-pocket2-generation.json'

with open(file_path, 'r') as f:
    data = json.load(f)

In [60]:
all_scores = generated_ds_exp_store['docking_scores'][:13800]

In [61]:
new_scores = all_scores[:12800] + data['14gs_A']['docking_scores']

In [62]:
print(np.mean(sorted(all_scores)[:100]))

-9.749460029602051


In [63]:
print(np.mean(sorted(new_scores)[:100]))

-9.752590017318726
